THIS FILE CONTAINS THE CODE FOR IMPUTING THE MISSING TRAINING DATA AND THE DATA CLEANING CODE

Downloading the data using the kaggle API

In [ ]:
from google.colab import files
file=files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp /content/kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c ashrae-energy-prediction

  0% 0.00/2.53M [00:00<?, ?B/s]
100% 2.53M/2.53M [00:00<00:00, 82.7MB/s]
 94% 83.0M/88.4M [00:00<00:00, 106MB/s]
100% 88.4M/88.4M [00:00<00:00, 139MB/s]
 95% 114M/120M [00:00<00:00, 112MB/s]  
100% 120M/120M [00:00<00:00, 129MB/s]
  0% 0.00/44.5k [00:00<?, ?B/s]
100% 44.5k/44.5k [00:00<00:00, 68.9MB/s]
 97% 162M/167M [00:01<00:00, 114MB/s] 
100% 167M/167M [00:01<00:00, 114MB/s]
  0% 0.00/1.27M [00:00<?, ?B/s]
100% 1.27M/1.27M [00:00<00:00, 183MB/s]


In [ ]:
!unzip /content/weather_train.csv.zip

Archive:  /content/weather_train.csv.zip
  inflating: weather_train.csv       


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle

In [ ]:
from scipy.stats import stats
import datetime as dt

In [ ]:
from tqdm import tqdm

**FUNCTION FOR REDUCING THE MEMORY USAGE**

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
df_train=pd.read_csv('train.csv')

In [ ]:
df_train_red=reduce_mem_usage(df_train,verbose=True)

Mem. usage decreased to 289.19 Mb (53.1% reduction)


In [ ]:
df_weather_train=pd.read_csv('weather_train.csv')

In [ ]:
df_weather_train_red=reduce_mem_usage(df_weather_train,verbose=True)

Mem. usage decreased to  3.07 Mb (68.1% reduction)


In [ ]:
df_building=pd.read_csv('building_metadata.csv')

In [ ]:
df_building_red=reduce_mem_usage(df_building,verbose=True)

Mem. usage decreased to  0.03 Mb (60.3% reduction)


In [ ]:
df_train_build=pd.merge(df_train_red,df_building_red,how='left',on=['building_id'])

In [ ]:
df_train_merge=pd.merge(df_train_build,df_weather_train_red,how='left',on=['site_id','timestamp'])

**CONVERTING THE TIMESTAMP TO DATETIME OBJECT**

In [ ]:
df_train_merge['timestamp']=pd.to_datetime(df_train_merge['timestamp'])

**HERE I AM CREATING A FUNCTION FOR DATA CLEANING**

1.   AT SITE 0 FOR BUILDING ID UPTO 104 THE ELECTRICITY READINGS WERE ZERO FOR 5 MONTHS STARTING FROM JANUARY WHICH INDICATES THAT PROPER READINGS HAVE BEEN STARTED TAKEN FROM 21th MAY.

2.   THERE ARE MANY BUILDINGS WHICH WHICH HAVE INCONSISTENT READINGS OR BUILDINGS WHICH ARE SHOWING VERY HIGH READINGS WHICH NEEDS TO BE FILTERED OUT.





In [ ]:
def filter_outliers(df):

  df.drop(index=df[(df['building_id']<=104) & (df['meter']==0) & (df['timestamp']<'2016-05-21')].index,inplace=True)
  df.drop(index=df[(df['building_id']==45) & (df['meter']==0) & (df['timestamp']<'2016-06')].index,inplace=True)
  df.drop(index=df[(df['building_id']==53) & (df['meter']==0)].index,inplace=True)#Removing Anamolous Building
  df.drop(index=df[(df['building_id']==1099) & (df['meter']==2)].index,inplace=True)#Removing Anamolous Building
  df.drop(index=df[(df['building_id']==1250) & (df['meter']==2)].index,inplace=True)#Removing Anamolous Building
  df.drop(index=df[(df['building_id']==1227) & (df['meter']==0)].index,inplace=True)#Removing Anamolous Building
  df.drop(index=df[(df['building_id']==1314) & (df['meter']==0)].index,inplace=True)#Removing Anamolous Building
  df.drop(index=df[(df['building_id']==1281) & (df['meter']==0)].index,inplace=True)#Removing Anamolous Building
  df.drop(index=df[(df['building_id']==279) & (df['meter']==3)].index,inplace=True)#Removing Anamolous Building
  df.drop(index=df[(df['building_id']==263) & (df['meter']==3)].index,inplace=True)#Removing Anamolous Building
  df.drop(index=df[(df['building_id']==287) & (df['meter']==3)].index,inplace=True)#Removing Anamolous Building
  df.drop(index=df[(df['building_id']==1018) & (df['meter']==1)].index,inplace=True)#Removing Anamolous Building
  df.drop(index=df[(df['building_id']==1022) & (df['meter']==1)].index,inplace=True)#Removing Anamolous Building
  

  return df
  

In [ ]:
df_train_merge_cleaned=filter_outliers(df_train_merge)#GETTING THE CLEANED DATAFRAME FROM THE ABOVE FUNCTION

In [ ]:
df_train_merge_cleaned[['year_built','air_temperature','dew_temperature','cloud_coverage','precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed']]=df_train_merge_cleaned[['year_built','air_temperature','dew_temperature','cloud_coverage','precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed']].astype(np.float32)

                                  **Checking the percentage of missing values**

In [ ]:
(df_train_merge_cleaned.isnull().sum()/df_train_merge_cleaned.shape[0])*100

building_id            0.000000
meter                  0.000000
timestamp              0.000000
meter_reading          0.000000
site_id                0.000000
primary_use            0.000000
square_feet            0.000000
year_built            61.010820
floor_count           82.361341
air_temperature        0.486579
cloud_coverage        43.647889
dew_temperature        0.504150
precip_depth_1_hr     18.948532
sea_level_pressure     6.191485
wind_direction         7.264534
wind_speed             0.724510
dtype: float64

**Dropping the floor count values as it was more than 80% mising**

In [ ]:
df_train_merge_cleaned.drop('floor_count',axis=1,inplace=True)
df_train_merge_cleaned.reset_index(inplace=True)

In [ ]:
df_train_merge_cleaned['day']=df_train_merge_cleaned['timestamp'].dt.day
df_train_merge_cleaned['month']=df_train_merge_cleaned['timestamp'].dt.month

**FROM HERE I WILL BE IMPTUING THE MISSING VALUES**

1.   HERE THE BUILDINGS ARE PRESENT AT DIFFERENT GEOGRAPHICAL LOCATIONS DUE TO WHICH THERE ARE VARIATIONS OVER THE WEATHER CONDITIONS.
2.   SO INSTEAD OF IMPUTING IT DIRECTLY BY TAKING THE MEAN HERE I AM GROUPING IT BY THE SITE ID MONTH AND DAY WHICH PROVIDES A MORE ACCURATE DATA FOR FILLING THE MISSING VALUE.



In [ ]:
cc_fill=df_train_merge_cleaned.groupby(['site_id','day','month'])['cloud_coverage'].median().reset_index()
cc_fill.rename(columns={'cloud_coverage':'cc_filler'},inplace=True)
cc_fill['cc_filler'].fillna(method='ffill',inplace=True)
df_train_merge_cleaned=df_train_merge_cleaned.merge(cc_fill,how='left',on=['site_id','day','month'])
df_train_merge_cleaned['cloud_coverage'].fillna(df_train_merge_cleaned['cc_filler'],inplace=True)
df_train_merge_cleaned.drop(labels=['cc_filler'],axis=1,inplace=True)

In [ ]:
wd_fill=df_train_merge_cleaned.groupby(['site_id','day','month'])['wind_direction'].median().reset_index()
wd_fill.rename(columns={'wind_direction':'wind_direction_filler'},inplace=True)
df_train_merge_cleaned=df_train_merge_cleaned.merge(wd_fill,how='left',on=['site_id','day','month'])
df_train_merge_cleaned['wind_direction'].fillna(df_train_merge_cleaned['wind_direction_filler'],inplace=True)
df_train_merge_cleaned.drop(labels=['wind_direction_filler'],axis=1,inplace=True)

In [ ]:
ws_fill=df_train_merge_cleaned.groupby(['site_id','day','month'])['wind_speed'].median().reset_index()
ws_fill.rename(columns={'wind_speed':'wind_speed_filler'},inplace=True)
df_train_merge_cleaned=df_train_merge_cleaned.merge(ws_fill,how='left',on=['site_id','day','month'])
df_train_merge_cleaned['wind_speed'].fillna(df_train_merge_cleaned['wind_speed_filler'],inplace=True)
df_train_merge_cleaned.drop(labels=['wind_speed_filler'],axis=1,inplace=True)

In [ ]:
slp_fill=df_train_merge_cleaned.groupby(['site_id','day','month'])['sea_level_pressure'].median().reset_index()
slp_fill.rename(columns={'sea_level_pressure':'slp_filler'},inplace=True)
slp_fill.fillna(method='ffill',inplace=True)
df_train_merge_cleaned=df_train_merge_cleaned.merge(slp_fill,how='left',on=['site_id','day','month'])
df_train_merge_cleaned['sea_level_pressure'].fillna(df_train_merge_cleaned['slp_filler'],inplace=True)
df_train_merge_cleaned.drop(labels=['slp_filler'],axis=1,inplace=True)

In [ ]:
pd_fill=df_train_merge_cleaned.groupby(['site_id','day','month'])['precip_depth_1_hr'].median().reset_index()
pd_fill.rename(columns={'precip_depth_1_hr':'pd_filler'},inplace=True)
pd_fill['pd_filler'].fillna(method='ffill',inplace=True)
df_train_merge_cleaned=df_train_merge_cleaned.merge(pd_fill,how='left',on=['site_id','day','month'])
df_train_merge_cleaned['precip_depth_1_hr'].fillna(df_train_merge_cleaned['pd_filler'],inplace=True)
df_train_merge_cleaned.drop(labels=['pd_filler'],axis=1,inplace=True)

In [ ]:
yb_fill=df_train_merge_cleaned.groupby(['site_id','day','month'])['year_built'].median().reset_index()
yb_fill.rename(columns={'year_built':'yb_filler'},inplace=True)
yb_fill['yb_filler'].fillna(method='ffill',inplace=True)
df_train_merge_cleaned=df_train_merge_cleaned.merge(yb_fill,how='left',on=['site_id','day','month'])
df_train_merge_cleaned['year_built'].fillna(df_train_merge_cleaned['yb_filler'],inplace=True)
df_train_merge_cleaned.drop(labels=['yb_filler'],axis=1,inplace=True)

**FOR AIR TEMPERATURE AND DEW TEMPERATURE AS IT WAS SHOWING STRONG CORRELATION FROM THE PREVIOUS HOUR DATA THEREFORE I AM USING LINEAR INTERPOLTAION METHOD HERE.**

In [ ]:
df_train_merge_cleaned['air_temperature']=df_train_merge_cleaned['air_temperature'].interpolate(method='linear')
df_train_merge_cleaned['dew_temperature']=df_train_merge_cleaned['dew_temperature'].interpolate(method='linear')

In [ ]:
df_train_merge_cleaned=reduce_mem_usage(df_train_merge_cleaned,verbose=True)

Mem. usage decreased to 1094.24 Mb (37.0% reduction)


                           Storing the file cleaned and imputed file on my drive

In [ ]:
filename='df_train_merge_cleaned_imputed.txt'
my_file=open(filename,'wb')

In [ ]:
pickle.dump(df_train_merge_cleaned,my_file)
my_file.close()

In [ ]:
#### REF--->https://www.kaggle.com/aitude/ashrae-missing-weather-data-handling
#https://www.kaggle.com/gemartin/load-data-reduce-memory-usage